<a href="https://colab.research.google.com/github/AnnSenina/python_hse_2024/blob/main/notebooks/7_%D0%9E%D0%BF%D0%B8%D1%81%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описательные статистики: меры среднего и рассеяния


1.   Пропуски (удаление, заполнение)
2.   Меры среднего
3.   Проверка данных на нормальность
4.   Меры рассеяния
5.   Выбросы
6.   Стандартизация
7.   Визуализация seaborn


In [233]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from scipy.stats import shapiro
from sklearn import preprocessing

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/python_hse_2024/main/data/penguins.csv')
df

In [ ]:
df.info()

In [ ]:
df.dropna() # для всего датафрейма
#  df.dropna(subset=['столбец1', 'столбец2']) # надежнее
# по умолчанию .dropna(axis = 0) для удаления строк с пропусками, для столбцов axis = 1

# .dropna() создает копию df с изменениями
# чтобы изменения вступили в силу:
# df.dropna(inplace = True) # ИЛИ:
# df = df.dropna()

In [ ]:
# заполнение пропусков
df.fillna(0) # можно заполнить одинаковыми значениями - например, нулями
# аналогично, изменения не вступят в силу, копия df

### Меры среднего (= меры центральной тендаенции)

In [ ]:
df.describe()

In [ ]:
# для категориальных переменных
df[['species', 'island', 'sex']]

In [ ]:
df[['species', 'island', 'sex']].describe()
# если все переменные категориальные, мы автоматически видим их меры среднего и вариативности

In [ ]:
# можно также включить показ всех статистик
df.describe(include='all')

In [ ]:
# mean - среднее арифместическое - к столбцу можно вызвать метод df['столбец'].mean()
# 50% - медиана - к столбцу df['столбец'].median()
# top - мода - к столбцу df['столбец'].mode()[0] # Важно! мода возвращается списком, т.к. их может быть несколько

# давайте снова посмотрим на пропуски и заполним их мерой центральной тенденции
df.info()

In [ ]:
bill_l = df['bill_length_mm'].median()
df['bill_length_mm'] = df['bill_length_mm'].fillna(bill_l)
df.info()

In [ ]:
# заполните остальные пропуски


In [ ]:
# @title

df['bill_depth_mm'] = df['bill_depth_mm'].fillna(df['bill_depth_mm'].median())
df['flipper_length_mm'] = df['flipper_length_mm'].fillna(df['flipper_length_mm'].median())
df['body_mass_g'] = df['body_mass_g'].fillna(df['body_mass_g'].median())
df['sex'] = df['sex'].fillna(df['sex'].mode()[0])
df.info()

Немного более специфичная задача: пусть у нас есть датафрейм с велосипедами, в котором не везде заполнена температура воздуха

In [ ]:
bikes = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/Other/main/BikeDataVar.csv')
bikes.describe()

# смотрим на среднюю и медианную температуру

In [ ]:
# вариант 1: ищем медианную температуру для сезона:
bikes.groupby('Seasons')['Temperature'].median().sort_values()

In [ ]:
# .transform похож на .apply, однако он возвращает таблицу исходной длины
median_table = bikes.groupby('Seasons')['Temperature'].transform('median')
median_table
# вся таблица заполнена медианами по сезонам - нам это не нужно, но мы можем использовать эту таблицу для fillna!

In [ ]:
bikes['Temperature'] = bikes['Temperature'].fillna(median_table)
bikes # пропуски заполнены медианами по сезонам!

In [ ]:
bikes.describe()

In [ ]:
# вариант 2: используем метод интерполяции .interpolate()

# вернем назад исодную таблицу
bikes = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/Other/main/BikeDataVar.csv')
bikes.info()

In [ ]:
bikes['Temperature'].interpolate()
# пропуски будут заполнены на основании соседних ячеек

In [ ]:
bikes['Temperature'] = bikes['Temperature'].interpolate()
bikes['Temperature'].isna().sum() # пропуски отсутствуют

In [ ]:
bikes.describe()

## Основные описательные статистики
- максимум, минимум, размах
- среднее, мода, медиана

In [ ]:
df['body_mass_g'].max()

In [ ]:
df['body_mass_g'].min()

In [ ]:
df['body_mass_g'].max() - df['body_mass_g'].min() # размах = в экселе интервал

In [ ]:
df['body_mass_g'].mean()

In [ ]:
df['body_mass_g'].median()

In [ ]:
df['body_mass_g'].mode()

In [ ]:
# мода - едиснтвенная мера центральной тенденции для категориальных переменных
df['island'].mode()

In [ ]:
df['species'].mode()

In [ ]:
# все эти описательные статистики можно считать к сгруппированным данным:
df.groupby('species')['body_mass_g'].median()

сложности с модой...

Для сгруппированных данных нельзя напрямую посчитать моду

Но! Есть специальный метод .agg в pandas (агрегирование данных), который позволяет посчитать все, что угодно :)

Документация [здесь](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html)

А вот здесь есть еще одна [тетрадка с примерами](https://dfedorov.spb.ru/pandas/%D0%9E%D0%B1%D1%8A%D1%8F%D1%81%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B9%20Grouper%20%D0%B8%20Agg%20%D0%B2%20Pandas.html)

In [ ]:
df[df['island'] == 'Biscoe']['species'].value_counts() # можно посмотреть, пингвины каких видов живут на острове Biscoe

In [ ]:
df[df['species'] == 'Adelie']['island'].value_counts() # или посмотреть, на каких островах живут пингвины вида Adelie

In [ ]:
df.groupby('island')['species'].mode() # не сработает

In [ ]:
df.groupby('island')['species'].agg(pd.Series.mode) # пингвинов какого вида большинство на каждом острове

### Проверка на нормальность

In [ ]:
df['body_mass_g'].hist(bins=30);

In [ ]:
df['body_mass_g'].plot(kind='density');

In [136]:
# код для графика отсюда: https://github.com/capissimo/python-for-data-science/blob/master/ch01_Statistics.ipynb

def qqplot(xs):
    '''Квантильный график (график квантиль-квантиль, Q-Q plot)'''
    d = {0:sorted(scipy.stats.norm.rvs(loc=0, scale=1, size=len(xs))),
         1:sorted(xs)}
    pd.DataFrame(d).plot.scatter(0, 1, s=5, grid=True)
    plt.xlabel('Квантили теоретического нормального распределения')
    plt.ylabel('Квантили данных')
    plt.title ('Квантильный график', fontweight='semibold')

In [ ]:
qqplot(df['body_mass_g'])

[Гайд](https://habr.com/ru/articles/578754/) по Q-Q plot

In [ ]:
# тест на нормальность из scipy.stats
# H0: случайная величина распределена нормально
# H1: распределение не является нормальным
# Если p-значение меньше 0,05, мы отвергаем нулевую гипотезу

_, p = scipy.stats.normaltest(df['body_mass_g'])
alpha = 0.05
if p < alpha:  # H0: случайная величина распределена нормально
    print("Не нормальное распределение для alpha = ", alpha)
else:
    print("Нормальное распределение для alpha = ", alpha)

In [ ]:
# тест Шапиро-Уилка
_, p = shapiro(df['body_mass_g'])

alpha = 0.05
if p < alpha:  # H0: случайная величина распределена нормально
    print("Не нормальное распределение для alpha = ", alpha)
else:
    print("Нормальное распределение для alpha = ", alpha)


In [ ]:
shapiro(df['body_mass_g'])

In [ ]:
shapiro(df['body_mass_g']).pvalue < 0.05

## Меры вариативности (= меры разброса)

- стандартное отклонение, дисперсия, квартили, процентили для чисел
- количество уникальных категорий для категориальных переменных

In [ ]:
df['body_mass_g'].var() # для выборки
# отличия: значения для выборки будут несколько выше, чем для ГС
# это сделано искусствено, т.к. более типичные значения распределены близко к среднему, их много -> они чаще попадают в выборку
# считается, что любая выборка будет несколько недооценивать разнообразие данных
# хитрые математики придумали выход (делят не на объем выборки n, а на n - 1) -> искусственно повышают разнообразие (рассеяние, вариативность) выборки

In [ ]:
df['body_mass_g'].var(ddof=0) # для ГС

In [ ]:
df['body_mass_g'].std() # для выборки

In [ ]:
df['body_mass_g'].std(ddof=0) # для ГС

In [ ]:
c_var = df['body_mass_g'].std() / df['body_mass_g'].mean() * 100
# c_var
print(c_var, '%')
# однородная совокупность (коэффициент вариации, или относительное стандартное отклонение - формулка из презентации)

In [ ]:
df.describe()['body_mass_g']['75%'] - df.describe()['body_mass_g']['25%'] # межвартильное расстояние

In [ ]:
df.describe()['body_mass_g']['25%'] # так ищем квартили (25%, 50%, 75%)

In [ ]:
np.quantile(df['body_mass_g'], 0.25) # 25, 50, 75

In [ ]:
np.percentile(df['body_mass_g'], 90) # любой процентиль (1 квартиль = 25% процентиль)

Квартили мы видим на ящиках с усами

Нижний "ус" до начала ящика - первые 25% значений совокупности (= 1 квартиль), четверть самых маленьких значений в выборке (или ГС)

Ящик - половина всех значений, распределенные близко к медиане (2 и 3 квартили = от 25% до 75% процентиля)

Верзний "ус" - с 75% процентиля и выше (= 4 квартиль), четветь самых больших значений в выборке (или ГС)

Вне "усов" расположены выбросы

## Выбросы

In [ ]:
d = df.describe()['body_mass_g']['75%'] - df.describe()['body_mass_g']['25%']
d

In [ ]:
# выбросы справа
r = df.describe()['body_mass_g']['75%'] + 1.5 * d
# все значения больше r являются выбросами (сверху или справа)
r

In [ ]:
len(df[df['body_mass_g'] > r]) # выбросов справа нет

In [ ]:
# выбросы слева
l = df.describe()['body_mass_g']['25%'] - 1.5 * d
# все значения меньше l являются выбросами
l

In [ ]:
len(df[df['body_mass_g'] < l])

In [ ]:
# пример в одну ячейку для другого показателя
d = df.describe()['bill_length_mm']['75%'] - df.describe()['bill_length_mm']['25%']

print(df.describe()['bill_length_mm']['75%'] + 1.5 * d)
print(df.describe()['bill_length_mm']['25%'] - 1.5 * d)

print(len(df[df['bill_length_mm'] > df.describe()['bill_length_mm']['75%'] + 1.5 * d]), 'выбросов справа')
print(len(df[df['bill_length_mm'] < df.describe()['bill_length_mm']['25%'] - 1.5 * d]), 'выбросов слева')

In [ ]:
# визуализацией выбросов являются ящики с усами
df.boxplot();

# как исправить график:
# единицы измерения разные -> данные нужно стандартизировать

## Стандартизация

В sklearn есть несколько вариантов стандартизации. Один из самых распространенных и быстрых: StandardScalar: после масштабирования данные имеют нулевое среднее значение и единичную дисперсию

Другие: например, MinMaxScaler, normalize

[Документация](https://scikit-learn.ru/6-3-preprocessing-data/)

In [ ]:
df_num = df.select_dtypes(include=['int64', 'float64'])
df_num

In [ ]:
stand = preprocessing.StandardScaler()
stand.fit(df_num)

X = stand.transform(df_num)
# x - это матрица класса numpy.ndarray
# преобразуем её в DataFrame
x = pd.DataFrame(X, index=df_num.index, columns=df_num.columns)
x

In [ ]:
round(x.mean(axis=0))

In [ ]:
round(x.std(axis=0))

In [ ]:
# можно собрать в функцию, чтобы дальше было удобнее работать
# здесь вариант покороче
def scale_features(df):
    scaled = preprocessing.StandardScaler().fit_transform(df)
    scaled = pd.DataFrame(scaled, columns=df.columns)
    return scaled

# пример:
scale_features(df_num)

In [ ]:
standart(df_num).boxplot();

Задание:
- выберите только числовые столбцы из bikes
- стандартизируйте данные
- постройте ящики с усами для всех столбцов
- определите количество выбросов для признака ```Rental Count```
- постройте гистрограмму и ящик с усами только для этого столбца


In [ ]:
# ваш код


In [ ]:
# @title
bikes_num = bikes.select_dtypes(include=['int64', 'float64'])
scale_features(bikes_num).boxplot()
plt.xticks(rotation=45);

In [ ]:
# @title
d = bikes.describe()['Rental Count']['75%'] - bikes.describe()['Rental Count']['25%']

print(len(bikes[bikes['Rental Count'] > bikes.describe()['Rental Count']['75%'] + 1.5 * d]), 'выбросов справа')
print(len(bikes[bikes['Rental Count'] < bikes.describe()['Rental Count']['25%'] - 1.5 * d]), 'выбросов слева')

In [ ]:
# @title
plt.hist(bikes['Rental Count'], bins=30);

In [ ]:
# @title
plt.boxplot(bikes['Rental Count']);

### Преобразования данных

Часто переменную логарифмируют, если подозревают, что ее распределение логнормальное и в исходном виде повлияет на дальнейшее исследование

In [ ]:
data = bikes['Rental Count']
data_log = np.log(bikes['Rental Count'] + 1)

fig, axs = plt.subplots(nrows = 1, ncols = 2)

#create histograms
axs[0].hist(data)
axs[1].hist(data_log)

#add title to each histogram
axs[0].set_title('Original Data')
axs[1].set_title('Log-Transformed Data');

In [ ]:
alpha = 0.05
_, p = shapiro(np.log(bikes['Rental Count'] + 1))
if p < alpha:
  print("не является логнормальным для alpha = " + str(alpha))
else:
  print("является логнормальным на уровне " + str(alpha))